# Preparing Experiments

In this notebook, we will prepare several parts of our experiments including
deploying and configuring our workload, loading traces, and creating functions
to be called to generate the workloads.

In [1]:
from tqdm.auto import tqdm
import itertools

import os

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

In [2]:
# making sure kn is set up correctly
!kn service ls

NAME            URL                                            LATEST                AGE     CONDITIONS   READY   REASON
autoscale-go    http://autoscale-go.default.kn.nima-dev.com    autoscale-go-00035    89d     3 OK / 3     True    
bench1          http://bench1.default.kn.nima-dev.com          bench1-00054          97d     3 OK / 3     True    
bentoml-iris    http://bentoml-iris.default.kn.nima-dev.com    bentoml-iris-00001    3m17s   3 OK / 3     True    
helloworld-go   http://helloworld-go.default.kn.nima-dev.com   helloworld-go-00001   99d     3 OK / 3     True    


In [3]:
EXP_CONFIG_NAME_DEFAULT = 'bentoml-iris-250m-512mb'
exp_config_name = os.getenv("DEPLOYMENT_CONFIG_NAME", EXP_CONFIG_NAME_DEFAULT)
exp_file = f"deployments/{exp_config_name}.json"

workload_spec = util.load_json_file(exp_file)

kn_command = kube.get_kn_command(**workload_spec)
print(kn_command)
# to run the command, we can simply use:
# !{kn_command}
# or from python:
# get_ipython().system('{kn_command}')

kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=512Mi' \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency


In [4]:
# deploy the workload
!{kn_command}

Applying service 'bentoml-iris' in namespace 'default':

  0.046s The Configuration is still working to reflect the latest desired specification.
 37.554s Traffic is not yet migrated to the latest revision.
 37.649s Ingress has not yet been reconciled.
 37.674s Waiting for load balancer to be ready
 37.912s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00002' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
